In [8]:
import os
import sys
import warnings
from distutils.dir_util import copy_tree

import bentoml
import mlflow
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline

from src.common.constants import ARTIFACT_PATH, DATA_PATH, LOG_FILEPATH
from src.common.logger import (
    handle_exception,
    log_feature_importance,
    set_logger,
)
from src.common.metrics import rmse_cv_score
from src.common.utils import get_param_set
from src.preprocess import preprocess_pipeline

/workspaces/lgcns-bike-sharing/.venv/lib/python3.9/site-packages/sklearn/preprocessing/_function_transformer.py:343: UserWarning: With transform="pandas", `func` should return a DataFrame to follow the set_output API.
  warnings.warn(


In [5]:
import os, sys
sys.path.insert(0,'../')

In [9]:
train_df = pd.read_csv(os.path.join(DATA_PATH, "bike_sharing_train.csv"))

In [11]:
train_df.head(3)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,32


In [14]:
train.columns

NameError: name 'train' is not defined

In [13]:
train_df.isna().sum()

datetime      0
season        0
holiday       0
workingday    0
weather       0
temp          0
atemp         0
humidity      0
windspeed     0
count         0
dtype: int64

In [12]:
train_df.describe()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count
count,5422.000000,5422.000000,5422.000000,5422.000000,5422.000000,5422.000000,5422.000000,5422.000000,5422.000000
mean,2.511988,0.026558,0.682036,1.426411,19.751988,23.157086,63.405017,12.924144,144.223349
std,1.114603,0.160804,0.465728,0.647407,7.951694,8.619132,19.780518,8.354137,133.312123
min,1.000000,0.000000,0.000000,1.000000,2.460000,3.030000,0.000000,0.000000,1.000000
25%,2.000000,0.000000,0.000000,1.000000,13.120000,15.910000,49.000000,7.001500,32.000000
50%,3.000000,0.000000,1.000000,1.000000,19.680000,23.485000,64.000000,12.998000,111.000000
75%,4.000000,0.000000,1.000000,2.000000,26.240000,31.060000,79.000000,19.001200,210.000000
max,4.000000,1.000000,1.000000,3.000000,38.540000,43.940000,100.000000,56.996900,638.000000
